# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="readonly/Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="readonly/Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [1]:
# Use the following data for this assignment:
%matplotlib notebook 

import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons, SpanSelector

import scipy.stats as stats

np.random.seed(12345)

colordict = { 1: 'red' , 2:'orange' , 3: 'white'  , 4:'lightblue' , 5:'blue'}

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])


nobs = df.shape[1]


## calculating mean, std error and confidence interval 

df['mean'] = df.mean(axis=1)
df['std'] = df.std(axis=1)
#std deviation of the mean is std deviation divided by sq root of nobs
df['std_mean'] = (df.std(axis = 1))/pow(nobs,0.5)
conf = df.apply(lambda x : stats.norm.interval(0.95, loc=x['mean'], scale=x['std_mean']),axis=1)
## conf values are a series of tuples, has to be unpacked. 
##### firt was of handling this, using a for loop 
####conf_left= (i[0] for i in conf.values)
## second way of handling this

df['conf_left'],df['conf_right'] = zip(*conf.values)
df['err_left'] = df['mean'] - df['conf_left']
df['err_right'] = df['conf_right'] - df['mean']
df['min'] = df.min(axis =1)
df['max'] = df.max(axis=1)




## retain only the useful columns
df = df[['mean','std','std_mean','conf_left','conf_right','err_left', 'err_right', 'min' , 'max']]

yerr = [df['conf_left'].values,df['conf_right'].values]


fig, ax = plt.subplots()  


## plotting the histogram with error bars






## function accepts a parameter and changes bar colors
def update_hist(y_min, y_max) :
    
    global df
    #df.loc[y_value <= df['conf_left'],'ycolor']=5
    #df.loc[((y_value > df['conf_left']) & (y_value < df['conf_right'])),'ycolor']=3
    #df.loc[(y_value > df['conf_right']),'ycolor']=1
    
    ##get confidence overlap
    
    df['range_probability'] = stats.norm.cdf(y_max, loc = df['mean'], scale = df['std_mean'] ) - stats.norm.cdf(y_min, loc = df['mean'], scale = df['std_mean'] )
    df['sign'] = np.where(y_min>df['mean'],-1,1) 
  
    
    df['color'] = df.apply(lambda x : plt.cm.Reds(1-x['range_probability']) if x['sign']== 1 
                           else plt.cm.Blues(1-x['range_probability']), axis =1)
 
    
    ## assign a y color
    
    #df['bar_color'] = df.apply(lambda x :x['sign'] )
    plt.bar(np.arange(0,4), df['mean'].values, yerr = [df['err_left'].values,df['err_right'].values], color=df['color'].values , edgecolor = 'black')
    x = plt.xticks(np.arange(0,4),df.index)
    plt.title('Interactive bar chart, drag the mouse to select a new y axis range, \n user has chosen the range %d - %d' %(y_min, y_max))
    fig.canvas.draw()
    fig1 = plt.gcf()
    fig1.savefig('interactive.png')
         
    

update_hist(40000,43000)

span = SpanSelector(ax, update_hist, 'vertical',useblit= True, rectprops=dict(alpha=0.3, facecolor='red'), span_stays = True)

plt.show()
    

ModuleNotFoundError: No module named 'numpy.testing.decorators'